In [10]:
parent_folder = 'C:/Users/Aidan/Documents/Winter_2023/BE177B/'
imP = parent_folder+'Paper_Figures/PolKymo.png'
imS = parent_folder+'Paper_Figures/SinKymo.png'
imR = parent_folder+'Paper_Figures/RandKymo.png'

In [11]:
import cv2
import numpy as np

In [12]:
def morph_it(im1, im2, frames, savename):
    # Load the two images
    img1 = cv2.imread(im1)
    img2 = cv2.imread(im2)

    # Define the number of intermediate frames
    num_frames = frames

    # Define the morphing points for each image
    src_points = np.array([[0, 0], [0, img1.shape[0]], [img1.shape[1], img1.shape[0]], [img1.shape[1], 0]], dtype=np.float32)
    int_points = np.array([[0, 0], [0, imgI.shape[0]], [imgI.shape[1], imgI.shape[0]], [imgI.shape[1], 0]], dtype=np.float32)
    dst_points = np.array([[0, 0], [0, img2.shape[0]], [img2.shape[1], img2.shape[0]], [img2.shape[1], 0]], dtype=np.float32)

    # Create a VideoWriter object to save the output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output = cv2.VideoWriter(savename+'.mp4', fourcc, 30.0, (img1.shape[1], img1.shape[0]))

    # Loop through each intermediate frame and morph between the two images
    for i in range(0,num_frames):
        # Calculate the intermediate points for the current frame
        alpha = float(i) / (num_frames - 1)
        points = (1 - alpha) * src_points + alpha * dst_points
        
        # Calculate the affine transform for the current frame
        transform = cv2.getPerspectiveTransform(src_points, points)
        
        # Apply the transform to the first image
        warped_img1 = cv2.warpPerspective(img1, transform, (img1.shape[1], img1.shape[0]))
        
        # Apply the transform to the second image
        warped_img2 = cv2.warpPerspective(img2, transform, (img2.shape[1], img2.shape[0]))
        
        # Blend the two images using alpha blending
        morphed_img = cv2.addWeighted(warped_img1, 1 - alpha, warped_img2, alpha, 0)
        
        # Write the current frame to the output
        output.write(np.uint8(morphed_img))
        
        # Display the current frame
        cv2.imshow('Morphing', morphed_img)
        
        # Wait for a key press
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    # Release the video writer and destroy the window
    output.release()
    cv2.destroyAllWindows()

In [13]:
morph_it(imP, imS, 200, 'Polar_to_Sin')
morph_it(imS, imP, 200, 'Sin_to_Polar')
morph_it(imR, imS, 200, 'Rand_to_Sin')
morph_it(imR, imP, 200, 'Rand_to_Polar')
morph_it(imS, imR, 200, 'Sin_to_Rand')
morph_it(imP, imR, 200, 'Polar_to_Rand')
